In [11]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
import random
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score

In [21]:
batch_size=256
epochs=300
seed=1
cuda=False and torch.cuda.is_available()
log_interval=10
r_dim=128
#z_dim=128
path_save="results_cnp_class/"

num_class=10
context_target_ratio=0.5 #0.7 context, 0.3 target

In [22]:

torch.manual_seed(seed)
random.seed(seed)
device = torch.device("cpu") #"cuda" if args.cuda else 

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)


In [4]:
def get_context_idx(N,batch_size):
    # generate the indeces of the N context points in a flattened image
    idx = random.sample(range(0, batch_size), N)
    idx = torch.tensor(idx, device=device)
    return idx

In [5]:
def generate_grid(h, w):
    rows = torch.linspace(0, 1, h, device=device)
    cols = torch.linspace(0, 1, w, device=device)
    grid = torch.stack([cols.repeat(h, 1).t().contiguous().view(-1), rows.repeat(w)], dim=1)
    grid = grid.unsqueeze(0)
    return grid

In [6]:
def idx_to_y(idx, data):
    # get the [0;1] pixel intensity at each index
    y = torch.index_select(data, dim=1, index=idx)
    return y


In [7]:
def idx_to_x(idx, batch_size):
    # From flat idx to 2d coordinates of the 28x28 grid. E.g. 35 -> (1, 7)
    
    x = torch.index_select(x_grid, dim=1, index=idx)
    
    x = x.expand(batch_size, -1, -1)
    
    return x

In [63]:
class CNPClass(nn.Module):
    def __init__(self, r_dim):#,z_dim
        super(CNPClass, self).__init__()
        self.r_dim = r_dim
        #self.z_dim = z_dim
        self.h_1 = nn.Linear(784+1, 256)
        self.h_2 = nn.Linear(256, 256)
        self.h_3 = nn.Linear(256, self.r_dim)

        self.g_1 = nn.Linear(num_class*self.r_dim + 784, 512)
        self.g_2 = nn.Linear(512,256)
        self.g_3 = nn.Linear(256,256)
        self.g_proba = nn.Linear(256, num_class)
        self.g_std = nn.Linear(256, num_class)
        

    def h(self, x_y):
        x_y = F.relu(self.h_1(x_y))
        x_y = F.relu(self.h_2(x_y))
        x_y = F.relu(self.h_3(x_y))
        return x_y

    def aggregate(self, r,y):
        #r is of size batch_size*r_dim
        r_list=[]
        y=y.view(y.shape[0])
        for i in range(num_class):
            
            r_i=r[y==i]
            if r_i.shape[0]==0:
                r_list.append(torch.zeros(r_dim))
            else:
                r_list.append(torch.mean(r_i, dim=0))
        return  torch.cat(r_list,dim=0)

    def g(self, rep, x_target):
        r_et_x = torch.cat([rep, x_target], dim=1)
        input = F.relu(self.g_1(r_et_x))
        input = F.relu(self.g_2(input))
        input = F.relu(self.g_3(input))
        proba=F.log_softmax(self.g_proba(input),dim=1)
        std=0.1 + 0.9 * F.softplus(self.g_proba(input))
        return proba,std

    def xy_to_r_params(self, x, y):

        x_y = torch.cat([x, y.float()], dim=1)
        r_i = self.h(x_y)        
        r = self.aggregate(r_i,y)
        
        return r

    def forward(self, x_context, y_context,x_target):
        r_context = self.xy_to_r_params(x_context, y_context)  # (mu, logvar) of z
        #expand r to pass in the MLP
        r_expand = r_context.expand(x_target.shape[0], -1)
        # reconstruct the whole image including the provided context points
        proba,std = self.g(r_expand, x_target)
        return proba,std
        

In [64]:
def np_loss(proba,std, y,criterion = nn.CrossEntropyLoss()):#, z_all, z_context
    #need improvement
    '''
    batchsize=y.shape[0]
    y=y.view(batchsize).long()
    loss=torch.tensor(0.)
    a=0
    for i in range(num_class):
        loss+=torch.sum(torch.log(proba[y==i,i]))
    '''
    label=torch.zeros_like(y)
    
    return criterion(proba, y)+torch.log(std).sum(dim=0).mean()

In [65]:
model = CNPClass(r_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
x_grid = generate_grid(28, 28)
os.makedirs(path_save, exist_ok=True)

In [66]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (img, label) in enumerate(train_loader):
        
        batch_size = img.shape[0]
        img=img.to(device).squeeze().view(batch_size,-1)
        label=label.to(device).view(batch_size,1)
        
        #context_idx = get_context_idx(int(context_target_ratio*batch_size),batch_size)
        #create context set
        #img_context = img[context_idx]
        #label_context = label[context_idx]
        
        
        proba,std= model(img, label,img)#img_context, label_context,
        
        optimizer.zero_grad()
        
        loss = np_loss(proba,std,label.view(batch_size))
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(label), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(label)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

In [67]:
def test(epoch):
    model.eval()
    test_loss = 0
    label_true=[]
    label_pred=[]
    with torch.no_grad():
        for batch_idx, (img, label) in enumerate(test_loader):
            batch_size = img.shape[0]
            img=img.to(device).squeeze().view(batch_size,-1)
            label=label.to(device).view(batch_size,1)

            #N = int(context_target_ratio*batch_size)
            #context_idx = get_context_idx(N,batch_size)
            #img_context = img[context_idx]
            #label_context = label[context_idx]
            proba,std= model(img, label,img)#_context
            label_true.append(label.view(batch_size))
            label_pred.append(torch.argmax(proba,dim=1).view(batch_size))     
            test_loss += criterion(proba,label.view(batch_size)) #z_all, z_context
            
    
    y_pred=torch.cat(label_pred,dim=0).numpy()
    y_true=torch.cat(label_true,dim=0).numpy()
    
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    print(confusion_matrix(y_true,y_pred))
    print("Accuracy:",accuracy_score(y_true,y_pred))

In [68]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: -0.316870
Train Epoch: 1 [2560/60000 (4%)]	Loss: -2.276237
Train Epoch: 1 [5120/60000 (9%)]	Loss: -2.239584
Train Epoch: 1 [7680/60000 (13%)]	Loss: -2.244142
Train Epoch: 1 [10240/60000 (17%)]	Loss: -2.280963
Train Epoch: 1 [12800/60000 (21%)]	Loss: -2.291675
Train Epoch: 1 [15360/60000 (26%)]	Loss: -2.292987
Train Epoch: 1 [17920/60000 (30%)]	Loss: -2.293345
Train Epoch: 1 [20480/60000 (34%)]	Loss: -2.293605
Train Epoch: 1 [23040/60000 (38%)]	Loss: -2.293623
Train Epoch: 1 [25600/60000 (43%)]	Loss: -2.293626
Train Epoch: 1 [28160/60000 (47%)]	Loss: -2.293646
Train Epoch: 1 [30720/60000 (51%)]	Loss: -2.293678
Train Epoch: 1 [33280/60000 (55%)]	Loss: -2.293697
Train Epoch: 1 [35840/60000 (60%)]	Loss: -2.293808
Train Epoch: 1 [38400/60000 (64%)]	Loss: -2.293777
Train Epoch: 1 [40960/60000 (68%)]	Loss: -2.293890
Train Epoch: 1 [43520/60000 (72%)]	Loss: -2.294044
Train Epoch: 1 [46080/60000 (77%)]	Loss: -2.293944
Train Epoch: 1 [48640/60000 (81%)]	Loss: 

Train Epoch: 5 [46080/60000 (77%)]	Loss: -2.300810
Train Epoch: 5 [48640/60000 (81%)]	Loss: -2.300922
Train Epoch: 5 [51200/60000 (85%)]	Loss: -2.301327
Train Epoch: 5 [53760/60000 (89%)]	Loss: -2.301129
Train Epoch: 5 [56320/60000 (94%)]	Loss: -2.300804
Train Epoch: 5 [58880/60000 (98%)]	Loss: -2.300399
====> Epoch: 5 Average loss: -2.3008
====> Test set loss: 0.0016
[[ 941    0    6    2    3    7   11    1    9    0]
 [   0 1106    2    6    0    1    5    0   14    1]
 [   8   13  906   13   14    1   20    6   47    4]
 [   1    3   31  857    1   50    1   15   39   12]
 [   2    8    2    0  851    0   12    1   10   96]
 [   7    2    8   76   18  712   16    4   37   12]
 [   7    3   10    0   18   19  897    0    4    0]
 [   2   29   12    2    6    0    0  851   13  113]
 [   6   12   13   13   18   33   10    6  829   34]
 [   7    3    0    5   48    7    0   23   16  900]]
Accuracy: 0.885
Train Epoch: 6 [0/60000 (0%)]	Loss: -2.300986
Train Epoch: 6 [2560/60000 (4%)]	Los

Train Epoch: 10 [2560/60000 (4%)]	Loss: -2.301138
Train Epoch: 10 [5120/60000 (9%)]	Loss: -2.301864
Train Epoch: 10 [7680/60000 (13%)]	Loss: -2.301525
Train Epoch: 10 [10240/60000 (17%)]	Loss: -2.301695
Train Epoch: 10 [12800/60000 (21%)]	Loss: -2.301901
Train Epoch: 10 [15360/60000 (26%)]	Loss: -2.301915
Train Epoch: 10 [17920/60000 (30%)]	Loss: -2.301425
Train Epoch: 10 [20480/60000 (34%)]	Loss: -2.301795
Train Epoch: 10 [23040/60000 (38%)]	Loss: -2.301424
Train Epoch: 10 [25600/60000 (43%)]	Loss: -2.301684
Train Epoch: 10 [28160/60000 (47%)]	Loss: -2.301541
Train Epoch: 10 [30720/60000 (51%)]	Loss: -2.301562
Train Epoch: 10 [33280/60000 (55%)]	Loss: -2.301571
Train Epoch: 10 [35840/60000 (60%)]	Loss: -2.301703
Train Epoch: 10 [38400/60000 (64%)]	Loss: -2.301674
Train Epoch: 10 [40960/60000 (68%)]	Loss: -2.301712
Train Epoch: 10 [43520/60000 (72%)]	Loss: -2.302106
Train Epoch: 10 [46080/60000 (77%)]	Loss: -2.301520
Train Epoch: 10 [48640/60000 (81%)]	Loss: -2.302015
Train Epoch: 10 [

Train Epoch: 14 [43520/60000 (72%)]	Loss: -2.302182
Train Epoch: 14 [46080/60000 (77%)]	Loss: -2.302054
Train Epoch: 14 [48640/60000 (81%)]	Loss: -2.301784
Train Epoch: 14 [51200/60000 (85%)]	Loss: -2.302125
Train Epoch: 14 [53760/60000 (89%)]	Loss: -2.301842
Train Epoch: 14 [56320/60000 (94%)]	Loss: -2.302065
Train Epoch: 14 [58880/60000 (98%)]	Loss: -2.302242
====> Epoch: 14 Average loss: -2.3020
====> Test set loss: 0.0006
[[ 948    0    3    2    0    7   17    0    2    1]
 [   0 1121    3    2    0    1    4    1    3    0]
 [   4    2  994    7    4    0    5    5   10    1]
 [   0    1    4  989    2    3    0    4    6    1]
 [   0    3    4    0  948    1   10    1    3   12]
 [   4    1    0   28    1  835   11    0    8    4]
 [   3    3    2    0   10    8  931    0    1    0]
 [   1   13   17   11    7    2    0  930    3   44]
 [   2    7    8   26   11    9   12    3  888    8]
 [   5    9    0   13   17    8    1    7    3  946]]
Accuracy: 0.953
Train Epoch: 15 [0/6000

Train Epoch: 19 [2560/60000 (4%)]	Loss: -2.302357
Train Epoch: 19 [5120/60000 (9%)]	Loss: -2.302212
Train Epoch: 19 [7680/60000 (13%)]	Loss: -2.302250
Train Epoch: 19 [10240/60000 (17%)]	Loss: -2.302335
Train Epoch: 19 [12800/60000 (21%)]	Loss: -2.302086
Train Epoch: 19 [15360/60000 (26%)]	Loss: -2.302364
Train Epoch: 19 [17920/60000 (30%)]	Loss: -2.302335
Train Epoch: 19 [20480/60000 (34%)]	Loss: -2.302244
Train Epoch: 19 [23040/60000 (38%)]	Loss: -2.302318
Train Epoch: 19 [25600/60000 (43%)]	Loss: -2.302337
Train Epoch: 19 [28160/60000 (47%)]	Loss: -2.302349
Train Epoch: 19 [30720/60000 (51%)]	Loss: -2.302161
Train Epoch: 19 [33280/60000 (55%)]	Loss: -2.302335
Train Epoch: 19 [35840/60000 (60%)]	Loss: -2.302330
Train Epoch: 19 [38400/60000 (64%)]	Loss: -2.302311
Train Epoch: 19 [40960/60000 (68%)]	Loss: -2.302232
Train Epoch: 19 [43520/60000 (72%)]	Loss: -2.302111
Train Epoch: 19 [46080/60000 (77%)]	Loss: -2.302310
Train Epoch: 19 [48640/60000 (81%)]	Loss: -2.301961
Train Epoch: 19 [

...

Train Epoch: 82 [0/60000 (0%)]	Loss: -2.302586
Train Epoch: 82 [2560/60000 (4%)]	Loss: -2.302586
Train Epoch: 82 [5120/60000 (9%)]	Loss: -2.302586
Train Epoch: 82 [7680/60000 (13%)]	Loss: -2.302586
Train Epoch: 82 [10240/60000 (17%)]	Loss: -2.302586
Train Epoch: 82 [12800/60000 (21%)]	Loss: -2.302586
Train Epoch: 82 [15360/60000 (26%)]	Loss: -2.302586
Train Epoch: 82 [17920/60000 (30%)]	Loss: -2.302586
Train Epoch: 82 [20480/60000 (34%)]	Loss: -2.302586
Train Epoch: 82 [23040/60000 (38%)]	Loss: -2.302586
Train Epoch: 82 [25600/60000 (43%)]	Loss: -2.302586
Train Epoch: 82 [28160/60000 (47%)]	Loss: -2.302586
Train Epoch: 82 [30720/60000 (51%)]	Loss: -2.302586
Train Epoch: 82 [33280/60000 (55%)]	Loss: -2.302586
Train Epoch: 82 [35840/60000 (60%)]	Loss: -2.302586
Train Epoch: 82 [38400/60000 (64%)]	Loss: -2.302586
Train Epoch: 82 [40960/60000 (68%)]	Loss: -2.302586
Train Epoch: 82 [43520/60000 (72%)]	Loss: -2.302586
Train Epoch: 82 [46080/60000 (77%)]	Loss: -2.302586
Train Epoch: 82 [48640

Train Epoch: 86 [40960/60000 (68%)]	Loss: -2.302586
Train Epoch: 86 [43520/60000 (72%)]	Loss: -2.302586
Train Epoch: 86 [46080/60000 (77%)]	Loss: -2.302586
Train Epoch: 86 [48640/60000 (81%)]	Loss: -2.302586
Train Epoch: 86 [51200/60000 (85%)]	Loss: -2.302586
Train Epoch: 86 [53760/60000 (89%)]	Loss: -2.302586
Train Epoch: 86 [56320/60000 (94%)]	Loss: -2.302586
Train Epoch: 86 [58880/60000 (98%)]	Loss: -2.302586
====> Epoch: 86 Average loss: -2.3026
====> Test set loss: 0.0007
[[ 970    1    0    1    0    1    2    1    3    1]
 [   0 1127    2    1    0    0    1    1    3    0]
 [   3    0 1012    4    2    0    1    4    6    0]
 [   0    0    4  989    0    3    0    4    4    6]
 [   1    0    1    0  965    0    7    1    1    6]
 [   1    0    0   12    1  871    3    1    3    0]
 [   3    3    2    0    6    5  937    0    2    0]
 [   1    7    9    1    1    0    0  998    2    9]
 [   3    1    4    4    3    4    3    2  945    5]
 [   2    2    0    4    9    2    1    5

Train Epoch: 91 [0/60000 (0%)]	Loss: -2.302586
Train Epoch: 91 [2560/60000 (4%)]	Loss: -2.302586
Train Epoch: 91 [5120/60000 (9%)]	Loss: -2.302586
Train Epoch: 91 [7680/60000 (13%)]	Loss: -2.302586
Train Epoch: 91 [10240/60000 (17%)]	Loss: -2.302586
Train Epoch: 91 [12800/60000 (21%)]	Loss: -2.302586
Train Epoch: 91 [15360/60000 (26%)]	Loss: -2.302586
Train Epoch: 91 [17920/60000 (30%)]	Loss: -2.302586
Train Epoch: 91 [20480/60000 (34%)]	Loss: -2.302586
Train Epoch: 91 [23040/60000 (38%)]	Loss: -2.302586
Train Epoch: 91 [25600/60000 (43%)]	Loss: -2.302586
Train Epoch: 91 [28160/60000 (47%)]	Loss: -2.302586
Train Epoch: 91 [30720/60000 (51%)]	Loss: -2.302586
Train Epoch: 91 [33280/60000 (55%)]	Loss: -2.302586
Train Epoch: 91 [35840/60000 (60%)]	Loss: -2.302586
Train Epoch: 91 [38400/60000 (64%)]	Loss: -2.302586
Train Epoch: 91 [40960/60000 (68%)]	Loss: -2.302586
Train Epoch: 91 [43520/60000 (72%)]	Loss: -2.302586
Train Epoch: 91 [46080/60000 (77%)]	Loss: -2.302586
Train Epoch: 91 [48640

Train Epoch: 95 [40960/60000 (68%)]	Loss: -2.302586
Train Epoch: 95 [43520/60000 (72%)]	Loss: -2.302586
Train Epoch: 95 [46080/60000 (77%)]	Loss: -2.302586
Train Epoch: 95 [48640/60000 (81%)]	Loss: -2.302586
Train Epoch: 95 [51200/60000 (85%)]	Loss: -2.302586
Train Epoch: 95 [53760/60000 (89%)]	Loss: -2.302586
Train Epoch: 95 [56320/60000 (94%)]	Loss: -2.302586


KeyboardInterrupt: 

In [69]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
h_1.weight 	 torch.Size([256, 785])
h_1.bias 	 torch.Size([256])
h_2.weight 	 torch.Size([256, 256])
h_2.bias 	 torch.Size([256])
h_3.weight 	 torch.Size([128, 256])
h_3.bias 	 torch.Size([128])
g_1.weight 	 torch.Size([512, 2064])
g_1.bias 	 torch.Size([512])
g_2.weight 	 torch.Size([256, 512])
g_2.bias 	 torch.Size([256])
g_3.weight 	 torch.Size([256, 256])
g_3.bias 	 torch.Size([256])
g_proba.weight 	 torch.Size([10, 256])
g_proba.bias 	 torch.Size([10])
g_std.weight 	 torch.Size([10, 256])
g_std.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {140044872416496: {'step': 22319, 'exp_avg': tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00, -5.8337e-12],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  7.0658e-10],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00, -8.9056e-16],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...

In [70]:
#save the model
torch.save(model.state_dict(), path_save+"model_param.pt")